In [1]:
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os


from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [2]:
#Load data frame
bank_marketing = fetch_ucirepo(id=222)

In [3]:
df = pd.DataFrame(bank_marketing.data.features)
df['sub'] = bank_marketing.data.targets
#drop poutcome because it has a lot of missing values
df = df.drop(columns=['poutcome', 'contact'])
df

,age,job,marital,education,default,balance,housing,loan,day_of_week,month,duration,campaign,pdays,previous,sub
0,58,management,married,tertiary,no,2143,yes,no,5,may,261,1,-1,0,no
1,44,technician,single,secondary,no,29,yes,no,5,may,151,1,-1,0,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,76,1,-1,0,no
3,47,blue-collar,married,NaN,no,1506,yes,no,5,may,92,1,-1,0,no
4,33,NaN,single,NaN,no,1,no,no,5,may,198,1,-1,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,17,nov,977,3,-1,0,yes
45207,71,retired,divorced,primary,no,1729,no,no,17,nov,456,2,-1,0,yes
45208,72,retired,married,secondary,no,5715,no,no,17,nov,1127,5,184,3,yes
45209,57,blue-collar,married,secondary,no,668,no,no,17,nov,508,4,-1,0,no


In [4]:
#Drop NaN
df = df.dropna()
print(df.isna().sum())
df

age            0
job            0
marital        0
education      0
default        0
balance        0
housing        0
loan           0
day_of_week    0
month          0
duration       0
campaign       0
pdays          0
previous       0
sub            0
dtype: int64


,age,job,marital,education,default,balance,housing,loan,day_of_week,month,duration,campaign,pdays,previous,sub
0,58,management,married,tertiary,no,2143,yes,no,5,may,261,1,-1,0,no
1,44,technician,single,secondary,no,29,yes,no,5,may,151,1,-1,0,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,5,may,76,1,-1,0,no
5,35,management,married,tertiary,no,231,yes,no,5,may,139,1,-1,0,no
6,28,management,single,tertiary,no,447,yes,yes,5,may,217,1,-1,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,17,nov,977,3,-1,0,yes
45207,71,retired,divorced,primary,no,1729,no,no,17,nov,456,2,-1,0,yes
45208,72,retired,married,secondary,no,5715,no,no,17,nov,1127,5,184,3,yes
45209,57,blue-collar,married,secondary,no,668,no,no,17,nov,508,4,-1,0,no


In [5]:
df_std = df.copy()
num_columns = df_std.select_dtypes(include=[np.number])

scaler = MinMaxScaler()

num_columns_standardized = scaler.fit_transform(num_columns)
df_std[num_columns.columns] = num_columns_standardized
df_std

,age,job,marital,education,default,balance,housing,loan,day_of_week,month,duration,campaign,pdays,previous,sub
0,0.519481,management,married,tertiary,no,0.092259,yes,no,0.133333,may,0.053070,0.000000,0.000000,0.000000,no
1,0.337662,technician,single,secondary,no,0.073067,yes,no,0.133333,may,0.030704,0.000000,0.000000,0.000000,no
2,0.194805,entrepreneur,married,secondary,no,0.072822,yes,yes,0.133333,may,0.015453,0.000000,0.000000,0.000000,no
5,0.220779,management,married,tertiary,no,0.074901,yes,no,0.133333,may,0.028264,0.000000,0.000000,0.000000,no
6,0.129870,management,single,tertiary,no,0.076862,yes,yes,0.133333,may,0.044124,0.000000,0.000000,0.000000,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.428571,technician,married,tertiary,no,0.080293,no,no,0.533333,nov,0.198658,0.035088,0.000000,0.000000,yes
45207,0.688312,retired,divorced,primary,no,0.088501,no,no,0.533333,nov,0.092721,0.017544,0.000000,0.000000,yes
45208,0.701299,retired,married,secondary,no,0.124689,no,no,0.533333,nov,0.229158,0.070175,0.212156,0.010909,yes
45209,0.506494,blue-collar,married,secondary,no,0.078868,no,no,0.533333,nov,0.103294,0.052632,0.000000,0.000000,no


In [6]:
num_colums = ["marital", 'sub', 'housing', 'loan', 'default']
df_new = pd.get_dummies(df_std, columns=num_colums, drop_first=True)
df_new = df_new.rename(columns={'sub_yes': 'sub', 'housing_yes': 'housing', 'loan_yes': 'loan', 'default_yes': 'default'})
#df_new = df_new.drop(columns=['marital_divorced', 'marital_single', 'sub_no', 'sub_yes', 'housing_no', 'housing_yes', 'loan_yes', 'loan_no'])
df_new

,age,job,education,balance,day_of_week,month,duration,campaign,pdays,previous,marital_married,marital_single,sub,housing,loan,default
0,0.519481,management,tertiary,0.092259,0.133333,may,0.053070,0.000000,0.000000,0.000000,1,0,0,1,0,0
1,0.337662,technician,secondary,0.073067,0.133333,may,0.030704,0.000000,0.000000,0.000000,0,1,0,1,0,0
2,0.194805,entrepreneur,secondary,0.072822,0.133333,may,0.015453,0.000000,0.000000,0.000000,1,0,0,1,1,0
5,0.220779,management,tertiary,0.074901,0.133333,may,0.028264,0.000000,0.000000,0.000000,1,0,0,1,0,0
6,0.129870,management,tertiary,0.076862,0.133333,may,0.044124,0.000000,0.000000,0.000000,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.428571,technician,tertiary,0.080293,0.533333,nov,0.198658,0.035088,0.000000,0.000000,1,0,1,0,0,0
45207,0.688312,retired,primary,0.088501,0.533333,nov,0.092721,0.017544,0.000000,0.000000,0,0,1,0,0,0
45208,0.701299,retired,secondary,0.124689,0.533333,nov,0.229158,0.070175,0.212156,0.010909,1,0,1,0,0,0
45209,0.506494,blue-collar,secondary,0.078868,0.533333,nov,0.103294,0.052632,0.000000,0.000000,1,0,0,0,0,0


In [7]:
encoded_df = df_new.copy()
label_encoder = LabelEncoder()
encoded_df['job'] = label_encoder.fit_transform(encoded_df['job'])
encoded_df['education'] = label_encoder.fit_transform(encoded_df['education'])
encoded_df = encoded_df.drop(columns=['day_of_week', 'month'])
encoded_df

,age,job,education,balance,duration,campaign,pdays,previous,marital_married,marital_single,sub,housing,loan,default
0,0.519481,4,2,0.092259,0.053070,0.000000,0.000000,0.000000,1,0,0,1,0,0
1,0.337662,9,1,0.073067,0.030704,0.000000,0.000000,0.000000,0,1,0,1,0,0
2,0.194805,2,1,0.072822,0.015453,0.000000,0.000000,0.000000,1,0,0,1,1,0
5,0.220779,4,2,0.074901,0.028264,0.000000,0.000000,0.000000,1,0,0,1,0,0
6,0.129870,4,2,0.076862,0.044124,0.000000,0.000000,0.000000,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.428571,9,2,0.080293,0.198658,0.035088,0.000000,0.000000,1,0,1,0,0,0
45207,0.688312,5,0,0.088501,0.092721,0.017544,0.000000,0.000000,0,0,1,0,0,0
45208,0.701299,5,1,0.124689,0.229158,0.070175,0.212156,0.010909,1,0,1,0,0,0
45209,0.506494,1,1,0.078868,0.103294,0.052632,0.000000,0.000000,1,0,0,0,0,0


In [8]:
encoded_df.to_parquet('../dataset/preprocessed_bank_marketing.gzip', engine="pyarrow", compression='gzip')
pd.read_parquet('preprocessed_bank_marketing.gzip')

,age,job,education,balance,duration,campaign,pdays,previous,marital_married,marital_single,sub,housing,loan,default
0,0.519481,4,2,0.092259,0.053070,0.000000,0.000000,0.000000,1,0,0,1,0,0
1,0.337662,9,1,0.073067,0.030704,0.000000,0.000000,0.000000,0,1,0,1,0,0
2,0.194805,2,1,0.072822,0.015453,0.000000,0.000000,0.000000,1,0,0,1,1,0
5,0.220779,4,2,0.074901,0.028264,0.000000,0.000000,0.000000,1,0,0,1,0,0
6,0.129870,4,2,0.076862,0.044124,0.000000,0.000000,0.000000,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,0.428571,9,2,0.080293,0.198658,0.035088,0.000000,0.000000,1,0,1,0,0,0
45207,0.688312,5,0,0.088501,0.092721,0.017544,0.000000,0.000000,0,0,1,0,0,0
45208,0.701299,5,1,0.124689,0.229158,0.070175,0.212156,0.010909,1,0,1,0,0,0
45209,0.506494,1,1,0.078868,0.103294,0.052632,0.000000,0.000000,1,0,0,0,0,0
